## POS tagging using modified Viterbi - by Nurul Q Khan

### Data Preparation

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\khann\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
# reading the tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# first few tagged sentences
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Splitting into train and test (95:5) 
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:10])

3718
196
[[('``', '.'), ('Over', 'ADP'), ('the', 'DET'), ('summer', 'NOUN'), ('months', 'NOUN'), (',', '.'), ('there', 'DET'), ('has', 'VERB'), ('been', 'VERB'), ('a', 'DET'), ('slowing', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('rate', 'NOUN'), ('of', 'ADP'), ('new', 'ADJ'), ('orders', 'NOUN'), ('from', 'ADP'), ('the', 'DET'), ('computer', 'NOUN'), ('sector', 'NOUN'), (',', '.'), ('our', 'PRON'), ('primary', 'ADJ'), ('market', 'NOUN'), (',', '.'), ("''", '.'), ('said', 'VERB'), ('*T*-1', 'X'), ('Wilfred', 'NOUN'), ('J.', 'NOUN'), ('Corrigan', 'NOUN'), (',', '.'), ('chairman', 'NOUN'), ('and', 'CONJ'), ('chief', 'ADJ'), ('executive', 'ADJ'), ('officer', 'NOUN'), ('.', '.')], [('It', 'PRON'), ('would', 'VERB'), ('ease', 'VERB'), ('jail', 'NOUN'), ('overcrowding', 'NOUN'), ('while', 'ADP'), ('*-1', 'X'), ('preserving', 'VERB'), ('historic', 'ADJ'), ('structures', 'NOUN'), (',', '.'), ('the', 'DET'), ('company', 'NOUN'), ('says', 'VERB'), ('0', 'X'), ('*T*-2', 'X'), ('.', '.')], [('About'

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95584

In [6]:
# Tokens
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])

['``', 'Over', 'the', 'summer', 'months', ',', 'there', 'has', 'been', 'a']


In [7]:
# Vocabulary
V = set(tokens)
print(len(V))

12064


In [8]:
# No. of tags
T = set([pair[1] for pair in train_tagged_words])
print(len(T))
print(T)

12
{'ADP', 'PRON', 'NUM', 'VERB', 'ADJ', 'X', 'CONJ', 'DET', 'PRT', 'ADV', 'NOUN', '.'}


In [9]:
# Emission Probability
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [10]:
# Compute word given tag [Emission Probability]
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [11]:
# compute tag given tag: tag2(t2) given tag1 (t1) [Transition Probability]
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [12]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [14]:
tags_df.head()

,ADP,PRON,NUM,VERB,ADJ,X,CONJ,DET,PRT,ADV,NOUN,.
ADP,0.016971,0.069058,0.062120,0.008432,0.105988,0.035543,0.000747,0.325115,0.001494,0.013342,0.321486,0.039705
PRON,0.023354,0.007657,0.007274,0.483155,0.073124,0.093798,0.005360,0.009571,0.011868,0.034074,0.210184,0.040582
NUM,0.036228,0.001497,0.184431,0.018862,0.032335,0.209281,0.013772,0.002994,0.027844,0.002695,0.350898,0.119162
VERB,0.091057,0.035803,0.022706,0.168785,0.065174,0.217839,0.005192,0.134687,0.031618,0.081448,0.110896,0.034795
ADJ,0.077188,0.000657,0.021022,0.012153,0.067499,0.021022,0.017573,0.004927,0.010675,0.004927,0.696995,0.065364


In [15]:
len(train_tagged_words)

95584

### Build the vanilla Viterbi based POS tagger

In [16]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [17]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_set

[[('Japan', 'NOUN'),
  ("'s", 'PRT'),
  ('regulators', 'NOUN'),
  ('have', 'VERB'),
  ('since', 'ADP'),
  ('tightened', 'VERB'),
  ('controls', 'NOUN'),
  ('on', 'ADP'),
  ('index-related', 'ADJ'),
  ('stock', 'NOUN'),
  ('purchases', 'NOUN'),
  ('.', '.')],
 [('In', 'ADP'),
  ('the', 'DET'),
  ('coming', 'VERB'),
  ('decade', 'NOUN'),
  (',', '.'),
  ('analysts', 'NOUN'),
  ('say', 'VERB'),
  ('0', 'X'),
  ('*T*-2', 'X'),
  (',', '.'),
  ('U.S.-Japanese', 'ADJ'),
  ('relations', 'NOUN'),
  ('will', 'VERB'),
  ('be', 'VERB'),
  ('tested', 'VERB'),
  ('*-1', 'X'),
  ('as', 'ADP'),
  ('Tokyo', 'NOUN'),
  ('comes', 'VERB'),
  ('to', 'PRT'),
  ('terms', 'NOUN'),
  ('with', 'ADP'),
  ('its', 'PRON'),
  ('new', 'ADJ'),
  ('status', 'NOUN'),
  ('as', 'ADP'),
  ('the', 'DET'),
  ('region', 'NOUN'),
  ("'s", 'PRT'),
  ('economic', 'ADJ'),
  ('behemoth', 'NOUN'),
  ('.', '.')],
 [('``', '.'),
  ('A', 'DET'),
  ('lot', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('stocks', 'NOUN'),
  ('that', '

In [18]:
# tagging the test sentences using the vanilla veterbi
start = time.time()
tagged_seq_vanilla = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq_vanilla)

Time taken in seconds:  1427.631109714508
[('Japan', 'NOUN'), ("'s", 'PRT'), ('regulators', 'NOUN'), ('have', 'VERB'), ('since', 'ADP'), ('tightened', 'ADP'), ('controls', 'NOUN'), ('on', 'ADP'), ('index-related', 'ADP'), ('stock', 'NOUN'), ('purchases', 'NOUN'), ('.', '.'), ('In', 'ADP'), ('the', 'DET'), ('coming', 'VERB'), ('decade', 'NOUN'), (',', '.'), ('analysts', 'NOUN'), ('say', 'VERB'), ('0', 'X'), ('*T*-2', 'X'), (',', '.'), ('U.S.-Japanese', 'ADP'), ('relations', 'NOUN'), ('will', 'VERB'), ('be', 'VERB'), ('tested', 'VERB'), ('*-1', 'X'), ('as', 'ADP'), ('Tokyo', 'NOUN'), ('comes', 'VERB'), ('to', 'PRT'), ('terms', 'NOUN'), ('with', 'ADP'), ('its', 'PRON'), ('new', 'ADJ'), ('status', 'NOUN'), ('as', 'ADP'), ('the', 'DET'), ('region', 'NOUN'), ("'s", 'PRT'), ('economic', 'ADJ'), ('behemoth', 'ADP'), ('.', '.'), ('``', '.'), ('A', 'DET'), ('lot', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('stocks', 'NOUN'), ('that', 'ADP'), ('*T*-117', 'ADP'), ('have', 'VERB'), ('been', 'VERB'), 

In [19]:
# accuracy
check_vanilla = [i for i, j in zip(tagged_seq_vanilla, test_run_base) if i == j] 
accuracy_vanilla = len(check_vanilla)/len(tagged_seq_vanilla)
accuracy_vanilla

0.9023959151610369

In [20]:
# Finiding the incorrectly tagged cases
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_vanilla, test_run_base)) if j[0]!=j[1]]

In [37]:
incorrect_tagged_cases

[[('be', 'VERB'), (('relaunched', 'ADV'), ('relaunched', 'VERB'))],
 [(',', '.'), (('as', 'ADP'), ('as', 'ADV'))],
 [('as', 'ADV'), (('early', 'ADJ'), ('early', 'ADV'))],
 [(',', '.'), (('aims', 'ADV'), ('aims', 'VERB'))],
 [('*EXP*-1', 'X'), (("'s", 'PRT'), ("'s", 'VERB'))],
 [('always', 'ADV'), (('walking', 'ADV'), ('walking', 'VERB'))],
 [("'", 'PRT'), (('binders', 'ADV'), ('binders', 'NOUN'))],
 [('.', '.'), (('Propaganda', 'ADV'), ('Propaganda', 'NOUN'))],
 [('it', 'PRON'), (('enables', 'ADV'), ('enables', 'VERB'))],
 [('make', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('own', 'ADJ'), (('mind', 'VERB'), ('mind', 'NOUN'))],
 [('few', 'ADJ'), (('blocks', 'VERB'), ('blocks', 'NOUN'))],
 [('U.S.', 'NOUN'), (('ambassador', 'ADV'), ('ambassador', 'NOUN'))],
 [("'s", 'PRT'), (('residence', 'ADV'), ('residence', 'NOUN'))],
 [('guards', 'NOUN'), (('encircling', 'ADV'), ('encircling', 'VERB'))],
 [('had', 'VERB'), (('discarded', 'ADV'), ('discarded', 'VERB'))],
 [('their', 'PRON'), (('Uzi

### Solve the problem of unknown words

In [27]:
# importing PorterStemmer
from nltk.stem.porter import PorterStemmer

In [28]:
# Creating PorterStemmer object
stemmer = PorterStemmer()

In [29]:
# Creating a dictionary of stemmized words with the original POS tags
stemmized = {}
for item in train_tagged_words:
    stemmized[stemmer.stem(item[0])] = item[1]

In [30]:
# method to return pos tags based on different rules
def custom_pos_tagger(word):
    stemmed_word = stemmer.stem(word)
    # check if the input word exists in stemmized dict
    if stemmed_word in stemmized:
        return stemmized[stemmed_word]
    # check if the input word exists in lower_train dict
    elif word.lower() in lower_train:
        return lower_train[word.lower()]
    else:
        return POS_morphological(word)

In [31]:
# Creating a dictinoary by converting words to lower case
lower_train = {}
for item in train_tagged_words:
    lower_train[item[0].lower()] = item[1]

In [32]:
# Create a POS Tagging method based on morphological rules
def POS_morphological(word):
    suffix_1= ["ing", "ed", "es"]
    if (word.istitle() or word.isupper()):
        return "NOUN"
    elif word.endswith("ing") or word.endswith("ed"):
        return "VERB"
    elif word.lower() in ['a', 'an', 'the']:
        return "DET"
    else:
        return None

In [33]:
def Viterbi_modified(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        
        if word not in V:
            state_max = custom_pos_tagger(word)
            if state_max is None: # custom_pos tagger didn't work 
                p = []
                for tag in T:
                    if key == 0:
                        transition_p = tags_df.loc['.', tag]
                    else:
                        transition_p = tags_df.loc[state[-1], tag]
                    
                    state_probability = transition_p    
                    p.append(state_probability)
                
                pmax = max(p)
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
            
        else:
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission probability and state probability
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [34]:
# tagging the test sentences using 2nd modification
start = time.time()
tagged_seq_modified = Viterbi_modified(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq_modified)

Time taken in seconds:  1417.3910603523254
[('Japan', 'NOUN'), ("'s", 'PRT'), ('regulators', 'NOUN'), ('have', 'VERB'), ('since', 'ADP'), ('tightened', 'VERB'), ('controls', 'VERB'), ('on', 'ADP'), ('index-related', 'VERB'), ('stock', 'NOUN'), ('purchases', 'NOUN'), ('.', '.'), ('In', 'ADP'), ('the', 'DET'), ('coming', 'VERB'), ('decade', 'NOUN'), (',', '.'), ('analysts', 'NOUN'), ('say', 'VERB'), ('0', 'X'), ('*T*-2', 'X'), (',', '.'), ('U.S.-Japanese', 'NOUN'), ('relations', 'NOUN'), ('will', 'VERB'), ('be', 'VERB'), ('tested', 'VERB'), ('*-1', 'X'), ('as', 'ADP'), ('Tokyo', 'NOUN'), ('comes', 'VERB'), ('to', 'PRT'), ('terms', 'NOUN'), ('with', 'ADP'), ('its', 'PRON'), ('new', 'ADJ'), ('status', 'NOUN'), ('as', 'ADP'), ('the', 'DET'), ('region', 'NOUN'), ("'s", 'PRT'), ('economic', 'ADJ'), ('behemoth', 'NOUN'), ('.', '.'), ('``', '.'), ('A', 'DET'), ('lot', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('stocks', 'NOUN'), ('that', 'ADP'), ('*T*-117', 'NOUN'), ('have', 'VERB'), ('been', 'VE

In [35]:
# Accuracy
check_modified = [i for i, j in zip(tagged_seq_modified, test_run_base) if i == j] 
accuracy_modified = len(check_modified)/len(tagged_seq_modified)
accuracy_modified

0.9422623723487824

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [48]:
# Accuracy with vanilla viterbi
print("Accuracy from Vanilla viterbi " + str(round(accuracy_vanilla * 100, 2)) + "%")
print("Accuracy from Modified viterbi " + str(round(accuracy_modified * 100, 2)) + "%")

Accuracy from Vanilla viterbi 90.24%
Accuracy from Modified viterbi 94.23%


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [38]:
# Using the sample test sentense given in problem
sample_test_sentenses = ["Android is a mobile operating system developed by Google.",
                         "Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.",
                         "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
                         "Twitter is an online news and social networking service on which users post and interact with messages known as tweets.",
                         "Before entering politics, Donald Trump was a domineering businessman and a television personality.",
                         "The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.",
                         "This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.",
                         "Show me the cheapest round trips from Dallas to Atlanta",
                         "I would like to see flights from Denver to Philadelphia.",
                         "Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.",
                         "NASA invited social media users to experience the launch of ICESAT-2 Satellite."]
print(sample_test_sentenses)

['Android is a mobile operating system developed by Google.', 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.', "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.", 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.', 'Before entering politics, Donald Trump was a domineering businessman and a television personality.', 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.', 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.', 'Show me the cheapest round trips from Dallas to Atlanta', 'I would like to see flights from Denver to Philadelphia.', 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.', 'NASA invited social media users to experience the laun

In [39]:
nltk.download('punkt')
sample_words = []
# Tokenize into words
tokenized_sents = [word_tokenize(i) for i in sample_test_sentenses]
for i in tokenized_sents:
    for k in i:
        sample_words.append(k)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khann\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
# Tagging the sample sentences with vanilla viterbi
start = time.time()
tagged_seq = Viterbi(sample_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  57.189762115478516
[('Android', 'ADP'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'ADP'), ('.', '.'), ('Android', 'ADP'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'ADP'), ('worldwide', 'ADP'), ('on', 'ADP'), ('smartphones', 'ADP'), ('since', 'ADP'), ('2011', 'ADP'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'ADP'), ('.', '.'), ('Google', 'ADP'), ('and', 'CONJ'), ('Twitter', 'ADP'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'ADP'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'ADP'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'ADP'), ("'s", 'PRT'), ('firehose', 'ADP'), ('.', '.'), ('Twitter', 'ADP'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADP'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on

In [42]:
# Tagging the sample sentences with modified viterbi
start = time.time()
tagged_seq = Viterbi_modified(sample_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  39.67696762084961
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'DET'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'N

# We can see that words like Android, Google, Twitter, ICESAT-2 are now correctly tagged and the accuracy has imporved 90.23 to 94.22